## AndLink and EvaluationLink examples

In [ ]:
import common
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import bindlink
from opencog.bindlink import execute_atom

create atomspace and set it as default

In [ ]:
atomspace = AtomSpace()
initialize_opencog(atomspace)

### knowledge base

Color and size are attributes

In [ ]:
InheritanceLink(ConceptNode("size"), ConceptNode("attribute"))
InheritanceLink(ConceptNode("color"), ConceptNode("attribute"))

different types of color and size

In [ ]:
InheritanceLink(ConceptNode("small"), ConceptNode("size"))
InheritanceLink(ConceptNode("big"), ConceptNode("size"))
InheritanceLink(ConceptNode("green"), ConceptNode("color"))
InheritanceLink(ConceptNode("red"), ConceptNode("color"))
InheritanceLink(ConceptNode("black"), ConceptNode("color"))

animals which have some attributes defined

In [ ]:
InheritanceLink(ConceptNode("frog"), ConceptNode("small")).tv = TruthValue(0.8, 0.9)
InheritanceLink(ConceptNode("frog"), ConceptNode("green")).tv = TruthValue(0.7, 0.9)
InheritanceLink(ConceptNode("tiger"), ConceptNode("big")).tv = TruthValue(0.9, 0.9)
InheritanceLink(ConceptNode("tiger"), ConceptNode("red")).tv = TruthValue(0.6, 0.9)
InheritanceLink(ConceptNode("sparrow"), ConceptNode("red")).tv = TruthValue(0.1, 0.9)
InheritanceLink(ConceptNode("sparrow"), ConceptNode("small")).tv = TruthValue(0.8, 0.9)
InheritanceLink(ConceptNode("zebra"), ConceptNode("black")).tv = TruthValue(0.5, 0.9)
InheritanceLink(ConceptNode("zebra"), ConceptNode("small")).tv = TruthValue(0.38, 0.9)

Define that all animals are actually animals

In [ ]:
InheritanceLink(ConceptNode("frog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("frog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("tiger"), ConceptNode("animal"))
InheritanceLink(ConceptNode("tiger"), ConceptNode("animal"))
InheritanceLink(ConceptNode("sparrow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("sparrow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("zebra"), ConceptNode("animal"))
InheritanceLink(ConceptNode("zebra"), ConceptNode("animal"))

**Query example**  
let's select all animals and attributes, filtering by some threshold


first define function and EvaluationLink

In [ ]:
threshold = 0.6

def apply_threshold(atom):
    if atom.tv.mean < threshold:
        return TruthValue(0, 1)
    return TruthValue(1, 1)

EvaluationLink will accept link    X <- Z  
and return flag if it "matches"

In [ ]:
inherits_property = InheritanceLink(VariableNode("X"), VariableNode("Z"))

# all Variables(X and Z) used in EvaluationLink should be grounded somewhere else first!
# eval link which provides filtering
eval_threshold = EvaluationLink(
                    GroundedPredicateNode("py: apply_threshold"),
                    ListLink(inherits_property))

### now build the BindLink

Constrain that Variable "X" inherits from "item"

In [ ]:
select_items = InheritanceLink(VariableNode("X"), ConceptNode("animal"))

In [ ]:
# using NotLink
Z_is_not_item = NotLink(IdenticalLink(VariableNode("Z"), ConceptNode("animal")))

In [ ]:
and_link = AndLink(eval_threshold, inherits_property, Z_is_not_item, select_items)
bindlink_filter = BindLink(and_link,
                           inherits_property)


print(bindlink(atomspace, bindlink_filter))

## Exercise: remove Z_is_not_item from condition